# Simple Scan Line Beamforming Example
`vbeam` supports multiple backends _(at the time of writing they are: `"numpy"` ad `"jax"`)_. `"numpy"` is active by default, but it runs severly slow as its implementation relies on Python for-loops.

To speed things up we set the active backend to `"jax"`, because unlike `"numpy"`, `"jax"` uses vectorization and runs on the GPU.

In [ ]:
from vbeam.fastmath import backend_manager

backend_manager.active_backend = "jax"

# Importing Example Data
We can import data using the `vbeam.data_importers.import_pyuff` function. It takes the data in PyUff format and a `Scan` object and returns (among a few other things) a beamformer function.

In [ ]:
import numpy as np

from pyuff_ustb import Uff
from vbeam.beamformers import get_beamformer
from vbeam.data_importers import import_pyuff
from vbeam.scan import sector_scan, Scanlines
from vbeam.util.download import cached_download

# Download and read the channel data
data_url = "http://www.ustb.no/datasets/Verasonics_P2-4_parasternal_long_small.uff"
uff = Uff(cached_download(data_url))
channel_data = uff.read("/channel_data")

# Import the data
setup = import_pyuff(channel_data)

# Define and set a custom sector scan
scan_angles = np.array([wave.source.azimuth for wave in channel_data.sequence])
scan_depths = np.linspace(0e-3, 110e-3, 512)
setup.scan = sector_scan(scan_angles, scan_depths)
# Using Scanlines points optimizer makes the beamformer faster because it only has to 
# beamform one column (scanline) per transmitted wave
setup.points_optimizer = Scanlines()

# Get a default beamformer
beamformer = get_beamformer(setup)

# Setting Up the Beamformer Instance
The `Beamformer` object that was created when we imported the data can be used as just a regular function. Since all the needed data was provided in this example data we can call it without any arguments.

But first we shall prepare the beamformer by JIT-compiling it. JIT-compiling a function often makes it run much faster.

In [ ]:
import jax

jitted_beamformer = jax.jit(beamformer)


# How Fast is it?
With JAX we call `.block_until_ready()` on the result to ensure that the job has actually finished (if running on the GPU).

In [ ]:
import time

start_time = time.perf_counter()
result = jitted_beamformer().block_until_ready()
elapsed_time = time.perf_counter() - start_time
print(f"{elapsed_time:.2f} seconds")


# How Fast is it a Thousand Times?

In [ ]:
num_times = 1000
start_time = time.perf_counter()
# We can start 1000 jobs asynchronously
results = [jitted_beamformer() for _ in range(num_times)]
# Let's wait for them to finish on the GPU
[r.block_until_ready() for r in results]
elapsed_time = time.perf_counter() - start_time
print(f"{elapsed_time:.2f} seconds")


_Huh._

It seems like it is faster to run it a thousand times than just a single time?

No mystery: it is because the first time we call `jitted_beamformer` is when JIT-compilation occurs. This is when the code is optimized to run much faster, but it can take a while.

# Plot The Results
Let's plot the result to verify that it actually did something reasonable:

In [ ]:
import matplotlib.pyplot as plt

# The example data consists of 5 frames — let's plot the first one:
first_frame = result[0]
# We transpose the image by calling `first_frame.T` because matplotlib uses a different
# convention for x- and y-axis. It expectes the shape of an image to be (height, width)
# instead of (width, height), the latter being the convention in vbeam.
plt.imshow(first_frame.T, aspect="auto")
